In [226]:
import pandas as pd
import numpy as np

In [227]:
INPUT_FILE = '../answers/male_29.xlsx'
GROUP_NAME = 'male_29'
AGE_EXPECTED = 'до 29'
SEX_EXPECTED = 'Мужской'

In [228]:
df = pd.read_excel(INPUT_FILE)
df = df[df['Источник ответов'] == GROUP_NAME]
df.shape

(231, 36)

In [229]:
df.head().T

,0,1,2,3,4
ID,1021945080,1021944909,1021931728,1021930182,1021928778
Время создания,2021-12-16 11:45:56,2021-12-16 11:45:49,2021-12-16 11:35:14,2021-12-16 11:33:54,2021-12-16 11:32:37
Время изменения,2021-12-16 11:45:56,2021-12-16 11:45:49,2021-12-16 11:35:14,2021-12-16 11:33:54,2021-12-16 11:32:37
Ваш пол,Мужской,Мужской,Мужской,Мужской,Мужской
Сколько вам лет?,до 29,30-49,до 29,до 29,до 29
"Укажите тип населённого пункта, в котором вы проживаете.",крупный город (свыше 500 тыс. жителей),малый город (до 50 тыс. жителей),сельский населённый пункт,крупный город (свыше 500 тыс. жителей),крупный город (свыше 500 тыс. жителей)
Источник ответов,male_29,male_29,male_29,male_29,male_29
Кого бы вы НЕ хотели видеть своим соседом? / Наркозависимого,Наркозависимого,Наркозависимого,Наркозависимого,Наркозависимого,Наркозависимого
Кого бы вы НЕ хотели видеть своим соседом? / Человека другой расы,NaN,NaN,NaN,NaN,NaN
Кого бы вы НЕ хотели видеть своим соседом? / Гомосексуала,Гомосексуала,NaN,NaN,NaN,NaN


# Control 1: sex preference

In [230]:
# Map answers to numbers from 0 to 4
# 0 - uncertain
# 1 - full female preference
# 4 - full male preference
question_col = 'Согласны ли вы с утверждением: В целом, из мужчин получаются более хорошие политические лидеры, чем из женщин?'
reverse_question_col = 'Согласны ли вы с утверждением: Среди политических лидеров женщины более компетентны, чем мужчины'
range_question_col = '"Насколько вы согласны с утверждением (1 - полностью НЕ согласен, \n10 - полностью согласен)"" / Мужчины лучше справляются с ответственными должностями'


question_answers = ['Затрудняюсь ответить', 'Полностью не согласен', 'Скорее не согласен', 'Скорее согласен', 'Полностью согласен', ]
question_to_male_preference = {k: i for i, k in enumerate(question_answers)}
reverse_question_answers = ['Затрудняюсь ответить', 'Абсолютно не согласен', 'Скорее не согласен', 'Скорее согласен', 'Абсолютно согласен']
reverse_question_to_male_preference = {k: 5-i for i, k in enumerate(reverse_question_answers)}
reverse_question_to_male_preference['Затрудняюсь ответить'] = 0
range_question_to_male_pereference = df[range_question_col]//3 + 1

male_preference_question = df[question_col].apply(lambda x: question_to_male_preference[x])
male_preference_reverse_question = df[reverse_question_col].apply(lambda x: reverse_question_to_male_preference[x])
male_preference_range = range_question_to_male_pereference

is_uncertain_question = (df[question_col] == 'Затрудняюсь ответить')
is_uncertain_reverse_question = (df[reverse_question_col] == 'Затрудняюсь ответить')

# Control fails if person prefers one sex in one question, but other sex in another question
# If the person is uncertain then we can't control

sexism_control_df = df[[question_col, reverse_question_col, range_question_col]].copy()
sexism_control_df['male_preference_question'] = male_preference_question
sexism_control_df['male_preference_reverse_question'] = male_preference_reverse_question
sexism_control_df['male_preference_range'] = male_preference_range
sexism_control_df['is_uncertain_question'] = is_uncertain_question
sexism_control_df['is_uncertain_reverse_question'] = is_uncertain_reverse_question
sexism_control_df['distance_questions'] = (male_preference_question-male_preference_reverse_question).abs()
sexism_control_df['distance_range'] = (male_preference_question-male_preference_range).abs()
sexism_control_df['fails_uncertanity'] = (is_uncertain_question != is_uncertain_reverse_question)
sexism_control_df['fails_question'] = sexism_control_df['distance_questions'] > 2
sexism_control_df['fails_range'] = (~is_uncertain_question) & (sexism_control_df['distance_range'] > 2)
sexism_control_df['fails_sexism'] = sexism_control_df['fails_uncertanity'] | sexism_control_df['fails_question'] | sexism_control_df['fails_range']

In [231]:
# Visualize random rows
print('Failed sexism check', sexism_control_df[sexism_control_df.fails_sexism].shape)
sexism_control_df[sexism_control_df.fails_sexism].sample(5).T

Failed sexism check (46, 14)


,166,8,124,7,189
"Согласны ли вы с утверждением: В целом, из мужчин получаются более хорошие политические лидеры, чем из женщин?",Затрудняюсь ответить,Затрудняюсь ответить,Скорее согласен,Полностью не согласен,Скорее согласен
"Согласны ли вы с утверждением: Среди политических лидеров женщины более компетентны, чем мужчины",Скорее не согласен,Абсолютно не согласен,Затрудняюсь ответить,Затрудняюсь ответить,Затрудняюсь ответить
"""Насколько вы согласны с утверждением (1 - полностью НЕ согласен, \n10 - полностью согласен)"""" / Мужчины лучше справляются с ответственными должностями",2.0,1.0,5.0,1.0,5.0
male_preference_question,0,0,3,1,3
male_preference_reverse_question,3,4,0,0,0
male_preference_range,1.0,1.0,2.0,1.0,2.0
is_uncertain_question,True,True,False,False,False
is_uncertain_reverse_question,False,False,True,True,True
distance_questions,3,4,3,1,3
distance_range,1.0,1.0,1.0,0.0,1.0


# Control 2: religion

In [232]:
question_col = 'Согласны ли вы с утверждением: Религия играет важную роль в моей жизни.'
question_answers = ['Затрудняюсь ответить', 'Абсолютно не согласнен', 'Скорее не согласен',
                    'Скорее согласен', 'Абсолютно согласнен']
question_answers_to_religion_preference = {k: i for i, k in enumerate(question_answers)}
religion_preference_question = df[question_col].apply(lambda x: question_answers_to_religion_preference[x])

range_question_col = '"Насколько вы согласны с утверждением (1 - полностью НЕ согласен, \n10 - полностью согласен)"" / Религия играет важную роль в моей жизни'
religion_preference_range = df[range_question_col]//3 + 1

is_uncertain_question = (df[question_col] == 'Затрудняюсь ответить')

religion_control_df = df[[question_col, range_question_col]].copy()
religion_control_df['religion_preference_question'] = religion_preference_question
religion_control_df['religion_preference_range'] = religion_preference_range
religion_control_df['is_uncertain_question'] = is_uncertain_question
religion_control_df['distance_range'] = (religion_preference_question-religion_preference_range).abs()
religion_control_df['fails_question'] = (~is_uncertain_question) & (religion_control_df['distance_range'] > 1)
religion_control_df['fails_uncertanity'] = (is_uncertain_question) & ((religion_preference_range - 2.5).abs() >= 1)
religion_control_df['fails_religion'] = religion_control_df['fails_question'] | religion_control_df['fails_uncertanity']
print('Failed religion check')
religion_control_df[religion_control_df['fails_religion']].shape

Failed religion check


(7, 9)

In [233]:
religion_control_df[religion_control_df.fails_religion].head().T

,17,32,54,56,76
Согласны ли вы с утверждением: Религия играет важную роль в моей жизни.,Затрудняюсь ответить,Скорее не согласен,Затрудняюсь ответить,Скорее согласен,Затрудняюсь ответить
"""Насколько вы согласны с утверждением (1 - полностью НЕ согласен, \n10 - полностью согласен)"""" / Религия играет важную роль в моей жизни",1.0,10.0,10.0,1.0,1.0
religion_preference_question,0,2,0,3,0
religion_preference_range,1.0,4.0,4.0,1.0,1.0
is_uncertain_question,True,False,True,False,True
distance_range,1.0,2.0,4.0,2.0,1.0
fails_question,False,True,False,True,False
fails_uncertanity,True,False,True,False,True
fails_religion,True,True,True,True,True


In [234]:
religion_control_df.sample(5).T

,103,227,90,80,18
Согласны ли вы с утверждением: Религия играет важную роль в моей жизни.,Абсолютно не согласнен,Абсолютно не согласнен,Скорее не согласен,Затрудняюсь ответить,Скорее не согласен
"""Насколько вы согласны с утверждением (1 - полностью НЕ согласен, \n10 - полностью согласен)"""" / Религия играет важную роль в моей жизни",1.0,1.0,2.0,3.0,5.0
religion_preference_question,1,1,2,0,2
religion_preference_range,1.0,1.0,1.0,2.0,2.0
is_uncertain_question,False,False,False,True,False
distance_range,0.0,0.0,1.0,2.0,0.0
fails_question,False,False,False,False,False
fails_uncertanity,False,False,False,False,False
fails_religion,False,False,False,False,False


In [235]:
df['failed_sexism_check'] = sexism_control_df.fails_sexism
df['failed_religion_check'] = religion_control_df.fails_religion
df['invalid_age'] = (df['Сколько вам лет?'] != AGE_EXPECTED)
df['invalid_sex'] = (df['Ваш пол'] != SEX_EXPECTED)
df['failed_control'] = df['failed_sexism_check'] | df['failed_religion_check']

df[df['failed_control']].shape

(48, 41)

In [236]:
df.head().T

,0,1,2,3,4
ID,1021945080,1021944909,1021931728,1021930182,1021928778
Время создания,2021-12-16 11:45:56,2021-12-16 11:45:49,2021-12-16 11:35:14,2021-12-16 11:33:54,2021-12-16 11:32:37
Время изменения,2021-12-16 11:45:56,2021-12-16 11:45:49,2021-12-16 11:35:14,2021-12-16 11:33:54,2021-12-16 11:32:37
Ваш пол,Мужской,Мужской,Мужской,Мужской,Мужской
Сколько вам лет?,до 29,30-49,до 29,до 29,до 29
"Укажите тип населённого пункта, в котором вы проживаете.",крупный город (свыше 500 тыс. жителей),малый город (до 50 тыс. жителей),сельский населённый пункт,крупный город (свыше 500 тыс. жителей),крупный город (свыше 500 тыс. жителей)
Источник ответов,male_29,male_29,male_29,male_29,male_29
Кого бы вы НЕ хотели видеть своим соседом? / Наркозависимого,Наркозависимого,Наркозависимого,Наркозависимого,Наркозависимого,Наркозависимого
Кого бы вы НЕ хотели видеть своим соседом? / Человека другой расы,NaN,NaN,NaN,NaN,NaN
Кого бы вы НЕ хотели видеть своим соседом? / Гомосексуала,Гомосексуала,NaN,NaN,NaN,NaN


In [237]:
df.to_csv(f'../processed/{GROUP_NAME}.csv', index=False)